In [1]:
%pip install fastapi

Defaulting to user installation because normal site-packages is not writeable
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)

   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   -------------------- ------------------- 2/4 [starlette]
   -------------------- ------------------- 2/4 [starlette]
   -------------------- ------------------- 2/4 [starlette]
   -------------------- ------------------- 2/4 [starlette]
   -------------------- ------------------- 2/4 [starlette]
   -------------------- ------------------- 2/4 [starlette]
   -------------------- ------------------- 2/4 [star

In [3]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Optional
# In a real environment, you would use:
# import joblib
# import os
# import numpy as np
# import pandas as pd

# --- Configuration for the Simulated Prediction ---
# These constants now define the simple rules used inside the simulated model's predict method.
WHALE_SHARK_MAX_NORTH_LAT = 30.0  # Approx. 30 degrees North
WHALE_SHARK_MAX_SOUTH_LAT = -35.0 # Approx. 35 degrees South (South is negative)
LAT_STEP = 10.0 # Step size for latitude iteration (degrees)
LON_STEP = 10.0 # Step size for longitude iteration (degrees)

# --- Pydantic Data Models for Structured Output ---

class PredictionResult(BaseModel):
    """Defines the structure for a single location prediction."""
    latitude: float
    longitude: float
    model_output_label: int # New field to show the raw model output (0 or 1)
    prediction: str
    is_possible_habitat: bool
    reason: str

class GlobalPredictionResponse(BaseModel):
    """Defines the structure for the overall API response."""
    total_locations_checked: int
    possible_habitat_count: int
    results: List[PredictionResult]

# --- Simulated PKL Model Class ---

class SimulatedHabitatPredictor:
    """
    This class simulates the behavior of a real machine learning model
    loaded from a 'habitat_predictor.pkl' file.
    
    In a real scenario, this would be the actual model object loaded
    with joblib or pickle.
    """
    def __init__(self):
        print("Simulated model initialized. Ready to predict.")
        
    def predict(self, data: List[List[float]]) -> List[int]:
        """
        Simulates the model's prediction method. It expects a list of [lat, lon] pairs.
        Returns a list of prediction labels (1 = Possible Habitat, 0 = Unlikely).
        """
        predictions = []
        
        # The data argument is expected to be a list of lists, e.g., [[lat1, lon1], [lat2, lon2]]
        for lat, lon in data:
            # --- START: Actual Prediction Logic (Moved from old simulate_predict) ---
            is_possible_habitat = False
            
            if WHALE_SHARK_MAX_SOUTH_LAT <= lat <= WHALE_SHARK_MAX_NORTH_LAT:
                is_possible_habitat = True
                
                # Crude check to simulate avoiding major landmasses (Refinement from training)
                if (lon > -100 and lon < -50) and (lat > 20 and lat < 50): # North America
                    is_possible_habitat = False
                elif (lon > -10 and lon < 100) and (lat > -10 and lat < 40): # Afro-Eurasia
                    # Try to exclude Sahara/Arabia
                    if (lon > 10 and lon < 40) and (lat > 10 and lat < 30):
                         is_possible_habitat = False
                         
            # --- END: Actual Prediction Logic ---
            
            predictions.append(1 if is_possible_habitat else 0)
            
        return predictions

# --- Global Model Loading (Simulated) ---

# REAL SCENARIO: Check if the model file exists and load it.
# MODEL_PATH = 'habitat_predictor.pkl'
# if os.path.exists(MODEL_PATH):
#     LOADED_MODEL = joblib.load(MODEL_PATH)
# else:
#     # Fallback or raise error
#     LOADED_MODEL = SimulatedHabitatPredictor()

# SIMULATED SCENARIO: Use the placeholder model directly.
LOADED_MODEL = SimulatedHabitatPredictor()


# --- Core Prediction Logic (Using Model) ---

def model_predict(lat: float, lon: float) -> PredictionResult:
    """
    Feeds a single lat/long pair into the globally loaded model and interprets the result.
    """
    # 1. Prepare input data (A real model often expects a DataFrame or NumPy array)
    input_data = [[lat, lon]]
    
    # 2. Call the model's prediction method
    model_output = LOADED_MODEL.predict(input_data)[0] # Get the first (and only) prediction

    # 3. Interpret the model's output
    is_possible_habitat = model_output == 1
    
    if is_possible_habitat:
        prediction_str = "Possible Habitat (Warm Tropical/Temperate Zone)"
        reason_str = "Model predicted high likelihood (Output 1) based on latitude and excluding major landmasses."
    else:
        prediction_str = "Outside Known Range"
        reason_str = "Model predicted low likelihood (Output 0). Location is in polar, far temperate, or land areas."

    return PredictionResult(
        latitude=round(lat, 2),
        longitude=round(lon, 2),
        model_output_label=model_output,
        prediction=prediction_str,
        is_possible_habitat=is_possible_habitat,
        reason=reason_str
    )

# --- FastAPI Endpoint ---

@app.get("/predict_range", response_model=GlobalPredictionResponse)
async def get_global_predictions():
    """
    Iterates through global lat/long coordinates in a grid and returns the
    model's habitat prediction for each location.
    """
    all_results: List[PredictionResult] = []
    possible_habitat_count = 0
    total_locations_checked = 0

    # Iterate over the entire globe with the defined step size
    lat = -90.0
    while lat <= 90.0:
        lon = -180.0
        while lon <= 180.0:
            total_locations_checked += 1
            # Call the renamed and updated prediction function
            prediction = model_predict(lat, lon)
            all_results.append(prediction)

            if prediction.is_possible_habitat:
                possible_habitat_count += 1

            lon += LON_STEP
        lat += LAT_STEP

    return GlobalPredictionResponse(
        total_locations_checked=total_locations_checked,
        possible_habitat_count=possible_habitat_count,
        results=all_results
    )

# To run this file, save it as main.py and execute:
# uvicorn main:app --reload


Simulated model initialized. Ready to predict.


In [9]:
import json
from typing import List, Dict

# --- Configuration for the Prediction ---
# Defines the broad tropical/subtropical zone for Whale Shark habitat.
WHALE_SHARK_MAX_NORTH_LAT = 30.0  # Approx. 30 degrees North
WHALE_SHARK_MAX_SOUTH_LAT = -35.0 # Approx. 35 degrees South (South is negative)

# Step size for iterating over the globe (10 degrees for concise output)
LAT_STEP = 10.0
LON_STEP = 10.0

# Global counter for the number of grid points that are determined to be ocean.
ocean_grid_count = 0

class SimulatedHabitatPredictor:
    """
    This class contains the core logic, simulating a model loaded from a .pkl file.
    The logic now includes finer-grained exclusions to simulate a real model
    avoiding less productive, deep-ocean zones.
    """

    def is_land(self, lat: float, lon: float) -> bool:
        """
        Simulates a land/ocean check using hardcoded geographical boundaries.
        Returns True if the coordinates fall over a simulated landmass.
        """
        # 1. Exclude North America (approx.)
        if (-100 < lon < -50) and (20 < lat < 50):
            return True
        # 2. Exclude Afro-Eurasian landmass (approx.)
        elif (-10 < lon < 100) and (-10 < lat < 40):
            # Further check to exclude a central portion (e.g., Sahara/Arabia)
            if (10 < lon < 40) and (10 < lat < 30):
                 return True
        
        # If none of the simulated land conditions are met, it is considered ocean.
        return False

    def predict(self, data: List[List[float]]) -> List[int]:
        """
        Simulates the model's prediction method. It expects a list of [lat, lon] pairs.
        Returns a list of prediction labels (1 = Possible Habitat, 0 = Unlikely).
        """
        global ocean_grid_count
        predictions = []

        for lat, lon in data:
            is_possible_habitat = False

            # Check for land first
            is_over_land = self.is_land(lat, lon)
            
            # If it's ocean, increment the global counter
            if not is_over_land:
                ocean_grid_count += 1

                # 1. Primary Check: Latitudinal Band (35° S to 30° N)
                if WHALE_SHARK_MAX_SOUTH_LAT <= lat <= WHALE_SHARK_MAX_NORTH_LAT:
                    is_possible_habitat = True
                    
                    # --- Secondary Check: Simulated Cold/Far Ocean Area Refinements ---
                    
                    # Exclude far North Pacific/Alaska/Bering Sea region
                    if lat == WHALE_SHARK_MAX_NORTH_LAT and lon < -130:
                         is_possible_habitat = False
                    
                    # Exclude far Southern Ocean extensions
                    if lat == WHALE_SHARK_MAX_SOUTH_LAT:
                         if lon < 0 or lon > 150: 
                             is_possible_habitat = False

                    # --- Tertiary Check: Simulated Open Ocean Exclusion ---
                    # This simulates avoiding deep, low-nutrient areas far from coastal shelves.
                    if is_possible_habitat:
                        # Central Atlantic: Far from coasts, low productivity
                        if (-45 < lon < -25) and (-20 < lat < 20):
                            is_possible_habitat = False
                        
                        # Central Pacific: Vast area, low productivity
                        elif (170 < abs(lon) < 180) and (-20 < lat < 20): # Covers both East and West of 180
                             is_possible_habitat = False
                        
                        # Deep Eastern Pacific exclusion (away from Galapagos/Central American coast)
                        elif (-140 < lon < -110) and (-10 < lat < 10):
                             is_possible_habitat = False

            predictions.append(1 if is_possible_habitat else 0)

        return predictions

def run_global_prediction():
    """
    Performs the global grid search using the simulated model and prints the results.
    """
    # Reset the counter for a fresh run
    global ocean_grid_count
    ocean_grid_count = 0
    
    # 1. Simulate model loading
    predictor = SimulatedHabitatPredictor()
    print("--- Whale Shark (Rhincodon typus) Habitat Prediction ---")
    print(f"Simulated Model Loaded. Scanning with {LAT_STEP}° x {LON_STEP}° resolution.")

    all_results: List[Dict] = []
    
    # 2. Prepare grid iteration
    lat = -90.0
    while lat <= 90.0:
        lon = -180.0
        
        # Prepare batch input for the current latitude row
        input_data = []
        lon_coords = []

        while lon <= 180.0:
            input_data.append([lat, lon])
            lon_coords.append(lon)
            lon += LON_STEP
        
        # 3. Batch prediction using the simulated model
        # Note: The 'ocean_grid_count' is incremented inside predictor.predict()
        model_outputs = predictor.predict(input_data)
        
        # 4. Process and store results
        for i, model_output in enumerate(model_outputs):
            lat_coord = lat
            lon_coord = lon_coords[i]
            
            # Recalculate is_over_land here to use for the reason string
            is_over_land = predictor.is_land(lat_coord, lon_coord)
            
            is_possible_habitat = model_output == 1

            if is_possible_habitat:
                prediction_str = "Possible Habitat"
                reason_str = "Within tropical/warm temperate zone (Model Output: 1)"
            elif is_over_land:
                prediction_str = "Unlikely"
                reason_str = "Simulated Landmass Exclusion"
            else:
                prediction_str = "Unlikely"
                reason_str = "Outside defined range or cold-water exclusion (Model Output: 0)"


            result = {
                "latitude": round(lat_coord, 2),
                "longitude": round(lon_coord, 2),
                "model_output": model_output,
                "prediction": prediction_str,
                "reason": reason_str
            }
            all_results.append(result)

        lat += LAT_STEP

    # 5. Output summary
    possible_areas = [r for r in all_results if r['model_output'] == 1]
    
    possible_count = len(possible_areas)
    total_count = len(all_results)
    
    print("\n--- Summary ---")
    print(f"Total locations checked: {total_count}")
    print(f"Total ocean grid points checked: {ocean_grid_count}")
    print(f"Locations predicted as possible habitat: {possible_count}")
    print("\n--- First 10 Predicted Possible Areas ---")
    
    for result in possible_areas[:10]:
        print(f"  Lat: {result['latitude']:>5}, Lon: {result['longitude']:>6} -> {result['prediction']}")

    print("\nFull results saved to 'whale_shark_predictions.json'")
    
    # Save all results to a JSON file for easy viewing/analysis
    with open('whale_shark_predictions.json', 'w') as f:
        json.dump(all_results, f, indent=2)

if __name__ == "__main__":
    run_global_prediction()


--- Whale Shark (Rhincodon typus) Habitat Prediction ---
Simulated Model Loaded. Scanning with 10.0° x 10.0° resolution.

--- Summary ---
Total locations checked: 703
Total ocean grid points checked: 693
Locations predicted as possible habitat: 240

--- First 10 Predicted Possible Areas ---
  Lat: -30.0, Lon: -180.0 -> Possible Habitat
  Lat: -30.0, Lon: -170.0 -> Possible Habitat
  Lat: -30.0, Lon: -160.0 -> Possible Habitat
  Lat: -30.0, Lon: -150.0 -> Possible Habitat
  Lat: -30.0, Lon: -140.0 -> Possible Habitat
  Lat: -30.0, Lon: -130.0 -> Possible Habitat
  Lat: -30.0, Lon: -120.0 -> Possible Habitat
  Lat: -30.0, Lon: -110.0 -> Possible Habitat
  Lat: -30.0, Lon: -100.0 -> Possible Habitat
  Lat: -30.0, Lon:  -90.0 -> Possible Habitat

Full results saved to 'whale_shark_predictions.json'
